# Deploy a training pipeline

## Before you start

Run the cell below to verify that all the required packages are installed.

In [ ]:
pip install -r "../requirements.txt"

### Connect to your workspace

To connect to a workspace, we need identifier parameters - a subscription ID, resource group name, and workspace name. Since you're working with a compute instance, managed by Azure Machine Learning, you can use the default values to connect to the workspace.

In [ ]:
from dotenv import load_dotenv
import os
load_dotenv() # take environment variables from .env.


subscription = os.environ["SUBSCRIPTION"]
resource_group = os.environ["RESOURCE_GROUP"]
ws_name = os.environ["WORKSPACE_NAME"]

In [ ]:
from azure.ai.ml import MLClient
from azure.identity import DefaultAzureCredential

# authenticate
credential = DefaultAzureCredential()

# Get a handle to the workspace
ml_client = MLClient(
    credential=credential,
    subscription_id=subscription,
    resource_group_name=resource_group,
    workspace_name=ws_name,
)

Create Data store and upload training data

In [ ]:
from azure.ai.ml.entities import Data
from azure.ai.ml.constants import AssetTypes

data_path = "../data"
dataset_name = "diabetes-data-train"

patient_dataset = Data(
    path=data_path,
    type=AssetTypes.URI_FOLDER,
    description="Training data for diabetes prediction",
    name=dataset_name,
)
ml_client.data.create_or_update(patient_dataset)

In [ ]:
patient_dataset = ml_client.data.get(
    name="diabetes-data-train", label="latest"
)

print(patient_dataset)

Load components

In [ ]:
from azure.ai.ml import load_component
parent_dir = "../components/"

fix_missing_data_component = load_component(source=parent_dir + "fix-missing-data.yml")
normalize_data_component = load_component(source=parent_dir + "normalize-data.yml")
train_decision_tree_component = load_component(source=parent_dir + "train-decision-tree.yml")
train_logistic_regression_component = load_component(source=parent_dir + "train-logistic-regression.yml")




Build pipeline

In [ ]:
from azure.ai.ml import Input
from azure.ai.ml.constants import AssetTypes
from azure.ai.ml.dsl import pipeline

@pipeline()
def diabetes_classification(pipeline_job_input):
    clean_data = fix_missing_data_component(input_data=pipeline_job_input)
    normalized_data = normalize_data_component(input_data=clean_data.outputs.output_data)
    train_model_decision_tree = train_decision_tree_component(training_data=normalized_data.outputs.output_data)
    train_model_logistic_regression = train_logistic_regression_component(training_data=normalized_data.outputs.output_data)


    return {
        "pipeline_job_transformed_data": normalized_data.outputs.output_data,
        "pipeline_job_trained_model_decision_tree": train_model_decision_tree.outputs.model_output_decision_tree,
        "pipeline_job_trained_model_logistic_regression": train_model_logistic_regression.outputs.model_output_logistic_reg,
    }

pipeline_job = diabetes_classification(Input(type=AssetTypes.URI_FILE, path=patient_dataset.path))
print(pipeline_job)

In [ ]:

# set pipeline level compute
pipeline_job.settings.default_compute = "aml-cluster"
# set pipeline level datastore
pipeline_job.settings.default_datastore = "workspaceblobstore"

# print the pipeline job again to review the changes
print(pipeline_job)

In [ ]:
# submit job to workspace
pipeline_job = ml_client.jobs.create_or_update(
    pipeline_job, experiment_name="pipeline_diabetes_training"
)
pipeline_job

Go inside AML Studio and check which model performs better and replace `superior_model_node`

In [ ]:
# List all child jobs in the job
child_jobs = ml_client.jobs.list(parent_job_name=pipeline_job.name)
superior_model_node = "train_model_decision_tree"

# Traverse and download all the outputs of child job
for child_job in child_jobs:
    # print(child_job.properties)
    if child_job.display_name == superior_model_node:
        props = child_job.properties
        reused_run_id = props.get('azureml.reusedrunid')

        if reused_run_id:
            job_id = reused_run_id
        else:
            job_id = child_job.name
        break

print(job_id)

In [ ]:
from azure.ai.ml.entities import Model
from azure.ai.ml.constants import AssetTypes
from azure.core.exceptions import HttpResponseError

run_model = Model(
    path=f"azureml://datastores/workspaceartifactstore/paths/ExperimentRun/dcid.{job_id}/outputs/models/",
    name="decision_tree_diabetes_classifier",
    description="Model trained using decision tree classifier",
    type=AssetTypes.MLFLOW_MODEL,
)

# Attempt to create or update the model
try:
    ml_client.models.create_or_update(run_model)
except HttpResponseError as e:
    print(f"Error: {e}")
    print(f"Response: {e.response}")